In [1]:
%pip install quantstats
%pip install numpy
%pip install pandas
%pip install matplotlib
%pip install sklearn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
      
      More information is available at
      https://github.com/scikit-learn/sklearn-

In [2]:
import quantstats as qs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv
from sklearn.linear_model import LinearRegression

In [3]:
# Change day and round here
day = 0
round = 1
file_path = f"prices_round_{round}_day_{day}.csv"
path = f"./data/day_{day}/{file_path}"

In [4]:
# Get data
data = pd.read_csv(path, delimiter=";", parse_dates=True)
data

,day,timestamp,product,bid_price_1,bid_volume_1,bid_price_2,bid_volume_2,bid_price_3,bid_volume_3,ask_price_1,ask_volume_1,ask_price_2,ask_volume_2,ask_price_3,ask_volume_3,mid_price,profit_and_loss
0,0,0,AMETHYSTS,9998,1,9995.0,30.0,NaN,NaN,10005,30,NaN,NaN,NaN,NaN,10001.5,0.0
1,0,0,STARFRUIT,5036,30,NaN,NaN,NaN,NaN,5043,30,NaN,NaN,NaN,NaN,5039.5,0.0
2,0,100,AMETHYSTS,9996,1,9995.0,30.0,NaN,NaN,10002,6,10004.0,1.0,10005.0,30.0,9999.0,0.0
3,0,100,STARFRUIT,5041,4,5037.0,1.0,5036.0,30.0,5043,31,NaN,NaN,NaN,NaN,5042.0,0.0
4,0,200,AMETHYSTS,9996,1,9995.0,21.0,NaN,NaN,10004,1,10005.0,21.0,NaN,NaN,10000.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,0,999700,STARFRUIT,5053,1,5048.0,26.0,NaN,NaN,5054,2,5055.0,24.0,NaN,NaN,5053.5,0.0
19996,0,999800,STARFRUIT,5047,27,NaN,NaN,NaN,NaN,5052,1,5054.0,27.0,NaN,NaN,5049.5,0.0
19997,0,999800,AMETHYSTS,9995,27,NaN,NaN,NaN,NaN,10005,27,NaN,NaN,NaN,NaN,10000.0,0.0
19998,0,999900,STARFRUIT,5048,2,5047.0,21.0,NaN,NaN,5054,23,NaN,NaN,NaN,NaN,5051.0,0.0


In [24]:
parsed_data = data[['day', 'timestamp', 'product', 'mid_price']].copy()
parsed_data.drop(parsed_data[parsed_data["product"] == "AMETHYSTS"].index, inplace=True)
parsed_data

,day,timestamp,product,mid_price
1,0,0,STARFRUIT,5039.5
3,0,100,STARFRUIT,5042.0
5,0,200,STARFRUIT,5040.0
6,0,300,STARFRUIT,5040.5
9,0,400,STARFRUIT,5039.5
...,...,...,...,...
19990,0,999500,STARFRUIT,5052.5
19992,0,999600,STARFRUIT,5051.0
19995,0,999700,STARFRUIT,5053.5
19996,0,999800,STARFRUIT,5049.5


In [18]:
def calculate_price_index(p1, p2, p3, type):
    l = [p1, p2, p3]
    
    for i in range(len(l)):
        if not isinstance(l[i], int):
            del l[i]

    if type == 'bid':
        return l.index(max(l))
    else:
        return l.index(min(l))

In [25]:
lags = 4
for i in range(1, lags + 1):
    parsed_data[f'lag{i}'] = parsed_data['mid_price'].shift(i)

parsed_data.dropna(inplace=True)
parsed_data

,day,timestamp,product,mid_price,lag1,lag2,lag3,lag4
9,0,400,STARFRUIT,5039.5,5040.5,5040.0,5042.0,5039.5
11,0,500,STARFRUIT,5040.5,5039.5,5040.5,5040.0,5042.0
13,0,600,STARFRUIT,5040.5,5040.5,5039.5,5040.5,5040.0
14,0,700,STARFRUIT,5040.0,5040.5,5040.5,5039.5,5040.5
17,0,800,STARFRUIT,5039.5,5040.0,5040.5,5040.5,5039.5
...,...,...,...,...,...,...,...,...
19990,0,999500,STARFRUIT,5052.5,5051.5,5052.0,5051.5,5052.0
19992,0,999600,STARFRUIT,5051.0,5052.5,5051.5,5052.0,5051.5
19995,0,999700,STARFRUIT,5053.5,5051.0,5052.5,5051.5,5052.0
19996,0,999800,STARFRUIT,5049.5,5053.5,5051.0,5052.5,5051.5


In [26]:
x = parsed_data[['lag1', 'lag2', 'lag3', 'lag4']]
x

,lag1,lag2,lag3,lag4
9,5040.5,5040.0,5042.0,5039.5
11,5039.5,5040.5,5040.0,5042.0
13,5040.5,5039.5,5040.5,5040.0
14,5040.5,5040.5,5039.5,5040.5
17,5040.0,5040.5,5040.5,5039.5
...,...,...,...,...
19990,5051.5,5052.0,5051.5,5052.0
19992,5052.5,5051.5,5052.0,5051.5
19995,5051.0,5052.5,5051.5,5052.0
19996,5053.5,5051.0,5052.5,5051.5


In [27]:
y = parsed_data['mid_price']
y

9        5039.5
11       5040.5
13       5040.5
14       5040.0
17       5039.5
          ...  
19990    5052.5
19992    5051.0
19995    5053.5
19996    5049.5
19998    5051.0
Name: mid_price, Length: 9996, dtype: float64

In [37]:
model = LinearRegression()
model.fit(x, y)
pred = model.predict(x)
parsed_data["pred_price"] = pred
parsed_data

,day,timestamp,product,mid_price,lag1,lag2,lag3,lag4,pred_price
9,0,400,STARFRUIT,5039.5,5040.5,5040.0,5042.0,5039.5,5040.534608
11,0,500,STARFRUIT,5040.5,5039.5,5040.5,5040.0,5042.0,5040.408905
13,0,600,STARFRUIT,5040.5,5040.5,5039.5,5040.5,5040.0,5040.205844
14,0,700,STARFRUIT,5040.0,5040.5,5040.5,5039.5,5040.5,5040.368957
17,0,800,STARFRUIT,5039.5,5040.0,5040.5,5040.5,5039.5,5040.199437
...,...,...,...,...,...,...,...,...,...
19990,0,999500,STARFRUIT,5052.5,5051.5,5052.0,5051.5,5052.0,5051.754275
19992,0,999600,STARFRUIT,5051.0,5052.5,5051.5,5052.0,5051.5,5051.970766
19995,0,999700,STARFRUIT,5053.5,5051.0,5052.5,5051.5,5052.0,5051.712585
19996,0,999800,STARFRUIT,5049.5,5053.5,5051.0,5052.5,5051.5,5052.283323


In [38]:
print("Coefficients:", model.coef_)
print("Intercept:", model.intercept_)
print("Accuracy:", model.score(x, y))

Coefficients: [0.34608027 0.26269948 0.19565408 0.19213413]
Intercept: 17.363839324123546
Accuracy: 0.9857661405281359
